In [38]:
import Erdos_Renyi as er 
from pyhawkes.models import DiscreteTimeNetworkHawkesModelSpikeAndSlab
from pyhawkes.internals import parents
import json 
import pandas as pd 
from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Parameters
p = 0.5
dt_max=200
subset = 200
K = subset

In [3]:
# Real data
settings = json.load(open("settings.json","r"))
# Path to file
events_embd = settings["out_path"]+"concepts_date_embd.csv"
# load events
print("Reading in data")
events = pd.read_csv(events_embd,header=None)


Reading in data


In [4]:
L = events.head(subset).iloc[:,:-1].values
S_real = np.eye(subset,dtype=np.int)#events.head(subset)[300].values

In [51]:
def logistic(x): 
    return 1./(1+np.exp(-x))

def get_W(data):
    dim = len(data)
    W = np.zeros((dim,dim))
    
    for i in range(dim):
        for j in range(i+1,dim):
            W[i,j] = np.linalg.norm(data[i] - data[j])
    
    W += W.T
    W_norm = 100/np.linalg.norm(W) * W 

    return W_norm

array([[0.        , 0.78597979, 0.46716433, ..., 0.61455838, 0.45565424,
        0.6543672 ],
       [0.78597979, 0.        , 0.61603973, ..., 0.76472263, 0.62782137,
        0.69494905],
       [0.46716433, 0.61603973, 0.        , ..., 0.52943073, 0.25831987,
        0.39659319],
       ...,
       [0.61455838, 0.76472263, 0.52943073, ..., 0.        , 0.51750504,
        0.64618552],
       [0.45565424, 0.62782137, 0.25831987, ..., 0.51750504, 0.        ,
        0.48938854],
       [0.6543672 , 0.69494905, 0.39659319, ..., 0.64618552, 0.48938854,
        0.        ]])

In [45]:
ld_network = er.LatentDistanceAdjacencyModel(K=K, L = L, dim=2, v=None, alpha=1.0, beta=1.0,kappa=1.0,p = p)

#weight_model = er.SpikeAndSlabGammaWeights(model = ld_network, parallel_resampling=False)

In [52]:
W = get_W(L)
P = ld_network.P
A = np.ones((K,K))

ll = P < W
A = ll.astype(np.int)
A

array([[0, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0]])

In [55]:
P*1000

array([[5.00000000e+02, 5.67694726e-21, 6.13395126e-06, ...,
        6.14019743e-12, 1.53966322e-05, 7.71564485e-14],
       [5.67694726e-21, 5.00000000e+02, 5.24308600e-12, ...,
        9.87418676e-20, 1.47279809e-12, 6.71258058e-16],
       [6.13395126e-06, 5.24308600e-12, 5.00000000e+02, ...,
        2.83590926e-08, 3.07402734e+00, 1.20633984e-03],
       ...,
       [6.14019743e-12, 9.87418676e-20, 2.83590926e-08, ...,
        5.00000000e+02, 8.36579717e-08, 1.93986743e-13],
       [1.53966322e-05, 1.47279809e-12, 3.07402734e+00, ...,
        8.36579717e-08, 5.00000000e+02, 9.72294426e-07],
       [7.71564485e-14, 6.71258058e-16, 1.20633984e-03, ...,
        1.93986743e-13, 9.72294426e-07, 5.00000000e+02]])

In [12]:
class empty_model():
    def __init__(self,S):
        self.dt = 0
        self.K = S.shape[0]
        self.B = 0
        self.bias_model = None

In [ ]:
data  = parents.DiscreteTimeParents(model = None,S = S_real, T = S_real.shape[0],F=None)
weight_model.resample(data)
#A,W = weight_model.A,weight_model.W
# resample graph with new data 
ld_network.resample(data=[weight_model.A,weight_model.W])



In [ ]:
G = nx.from_numpy_matrix(self.A)
nx.draw(G,with_labels=True,with_edges=True)